In [2]:
import numpy as np
import torch.nn as nn
import torch
import pandas
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as udata
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(0)
np.random.seed(0)

In [37]:
trainpath = "cifar10-2classes-trainset"
#trainpath = "singan-train"
transformer = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset1 = datasets.ImageFolder(root=trainpath, transform=transformer)
trainset2 = datasets.ImageFolder(root="singan-train", transform=transformer)
trainset = udata.ConcatDataset([trainset1, trainset2])
#trainset.idx_to_class = {v: k for k, v in trainset.class_to_idx.items()}

In [22]:
print(trainset.idx_to_class)
print(trainset.class_to_idx)

{0: 'cat', 1: 'dog'}
{'cat': 0, 'dog': 1}


In [38]:
testpath = "cifar10-2classes-testset"
testset = datasets.ImageFolder(root=testpath, transform=transformer)
testset.idx_to_class = {v: k for k, v in testset.class_to_idx.items()}

In [39]:
print(testset.idx_to_class)
print(testset.class_to_idx)

{0: 'cat', 1: 'dog'}
{'cat': 0, 'dog': 1}


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [40]:
trainloader = udata.DataLoader(trainset, batch_size = 20, shuffle=True)
testloader = udata.DataLoader(testset, batch_size = 20, shuffle=True)

In [107]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


cnn = CNN()
cnn.to(device)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [98]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(cnn.parameters(), lr= 0.001)

In [99]:
def accuarcy(dataloader): 
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            #print(inputs.size())
            #print("labels: ", labels, labels.size(0))
            outputs = cnn(inputs)
            #print(outputs.data)
            #might change
            softmax = nn.Softmax(dim=1)
            probability = softmax(outputs.data)
            _, predicted = torch.max(probability, 1)
            #print("pred: ", predicted)
            total += labels.size(0)
            #print("total: ", total)
            correct += (predicted == labels).sum().item()
            #print("correct: ", correct)
            if total % 100 == 98:
                print(total)
    #print(total)    
    #print('Accuracy of the network on the train data: %d %%' % (
        #100 * correct / total))
    return 100 * correct / total

In [100]:
def train(cnn,num, epsilon=0.003, filename = None):
    pre_epoch = float("inf")
    for epoch in range(num):
        running_loss = 0.0
        total = 0 
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            #print(inputs.size())
            optimizer.zero_grad()
            outputs = cnn(inputs)
            #print(inputs.size(), outputs.size())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            total += 1
        test_accuarcy = accuarcy(testloader)
        train_accuarcy = accuarcy(trainloader)
        epoch_loss = running_loss / total
        print('epoch [%d] loss: %.3f' % (epoch + 1, epoch_loss), train_accuarcy, test_accuarcy)
        if filename != None:
            f = open(filename, 'a')
            f.write('epoch {} loss: {}, train_accuracy: {}, test_accuracy: {} \n'.format(epoch+1, epoch_loss, train_accuarcy, test_accuarcy))
        pre_epoch = epoch_loss
        
    print('Finished Training')
    if filename != None:
        f.close()

In [ ]:
train(cnn, 100)

In [120]:
PATH = './model/cnn_20_cd_e20.pth'
torch.save(cnn.state_dict(), PATH)

In [38]:
PATH = './model/cnn_batch1_cd.pth'
cnn = CNN()
cnn.load_state_dict(torch.load(PATH))
cnn.to(device)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [118]:
accuracy(trainloader)

41
Accuracy of the network on the train data: 85 %


In [119]:
accura()

98
198
298
398
498
598
698
798
898
998
1098
1198
1298
1398
1498
1598
1698
1798
1898
1985
Accuracy of the network on the test data: 50 %


In [108]:
def train_model(use_singan, dir_name):
    #load train set
    trainpath = "cifar10-2classes-trainset"
    transformer = transforms.Compose(
        [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    if use_singan:
        #trainpath = "singan-train"
        trainset1 = datasets.ImageFolder(root=trainpath, transform=transformer)
        trainset2 = datasets.ImageFolder(root="singan-train", transform=transformer)
        trainset = udata.ConcatDataset([trainset1, trainset2])
    else:
        trainset = datasets.ImageFolder(root=trainpath, transform=transformer)
    
    #load test set
    testpath = "cifar10-2classes-testset"
    testset = datasets.ImageFolder(root=testpath, transform=transformer)
    
    #set up device and train/test loader
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainloader = udata.DataLoader(trainset, batch_size = 20, shuffle=True)
    testloader = udata.DataLoader(testset, batch_size = 20, shuffle=True)
    
    #Build CNN
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 = nn.Conv2d(3, 16, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(16, 32, 5)
            self.fc1 = nn.Linear(32 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 2)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 32 * 5 * 5)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x


    cnn = CNN()
    cnn.to(device)
    
    return cnn 
    
    
    
   
    

In [ ]:
# run train model 20 times, assign each experiment with a different random seed and save the output to a file

import time
import os
for experiment_num in range(20):
    t = int(time.time())
    
    dir_name = str(t)
    print(dir_name)
    #make directory
    os.system("mkdir result")
    os.system("mkdir ./result/{}".format(dir_name))
    
    #set random seed
    torch.manual_seed(t)
    np.random.seed(t)
    
    #train model without singan-generated images
    use_singan = False
    cnn1 = train_model(use_singan, dir_name)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr= 0.001)
    
    #start training data and save the print out to a file
    
    if use_singan:
        filename = './result/{}/with_singan.txt'.format(dir_name)
    else:
        filename = './result/{}/without_singan.txt'.format(dir_name)
    train(cnn1, 10, filename = filename)
    
    
    #set random seed
    torch.manual_seed(t)
    np.random.seed(t)
    
    #train model with singan-generated images
    use_singan = True
    cnn1 = train_model(use_singan, dir_name)
    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.Adam(cnn.parameters(), lr= 0.001)
    
    #start training data and save the print out to a file
    
    if use_singan:
        filename = './result/{}/with_singan.txt'.format(dir_name)
    else:
        filename = './result/{}/without_singan.txt'.format(dir_name)
    train(cnn1, 10, filename = filename)

    
    
    
    
    
    